1. moltemplate-style
    1. translation, rotation, clone
    1. typify bondtype, angletype...
    1. 

2. amber

In [1]:
%load_ext autoreload
%autoreload 2

import molpy as mp
from pathlib import Path

In [2]:

# import molvis as mv

class Methane(mp.Struct):

    def __init__(self):
        super().__init__()
        C = self.def_atom(name="C", xyz=(0.1, 0, -0.07))
        H1 = self.def_atom(name="H1", xyz=(-0.1, 0, -0.07))
        H2 = self.def_atom(name="H2", xyz=(0., 0.1, 0.07))
        H3 = self.def_atom(name="H3", xyz=(0., -0.1, 0.07))
        self.def_bond(C, H1)
        self.def_bond(C, H2)
        self.def_bond(C, H3)

methane = Methane()
# mp.io.write_pdb("methane.pdb", methane)

In [3]:
class Ethane(mp.Struct):
    def __init__(self):
        super().__init__()
        ch3_1 = mp.builder.CH3()
        ch3_1.translate(-ch3_1["atoms"][0].xyz)
        ch3_2 = mp.builder.CH3()
        ch3_2.translate(-ch3_2["atoms"][0].xyz)
        ch3_2.rotate(180, axis=(0, 1, 0))
        self.add_struct(ch3_1)
        self.add_struct(ch3_2)
        self.def_bond(ch3_1["atoms"][0], ch3_2["atoms"][0])

ethane = Ethane()
# Get topology
topology = ethane.get_topology()

# Calculate angles and dihedrals
angles = ethane.calc_angles(topology)
dihedrals = ethane.calc_dihedrals(topology)

In [4]:
ff = mp.ForceField("opls")
atomstyle = ff.def_atomstyle("full")

# Define atom types
C_type = atomstyle.def_type("C3", charge=-0.18, sigma=3.5, epsilon=0.066)
H_type = atomstyle.def_type("HC", charge=0.06, sigma=2.5, epsilon=0.03)

# Define bond styles
bondstyle = ff.def_bondstyle("harmonic")
bondstyle.def_type(C_type, C_type, r0=1.54, k=310)
bondstyle.def_type(C_type, H_type, r0=1.09, k=340)

# Define angle styles
anglestyle = ff.def_anglestyle("harmonic")
anglestyle.def_type(H_type, C_type, H_type, theta0=109.5, k=33)
anglestyle.def_type(H_type, C_type, C_type, theta0=110.0, k=35)
anglestyle.def_type(C_type, C_type, C_type, theta0=112.0, k=40)

# Define dihedral styles
dihedralstyle = ff.def_dihedralstyle("opls")
dihedralstyle.def_type(H_type, C_type, C_type, H_type, c0=0.0, c1=1.0, c2=0.0, c3=-0.5)
dihedralstyle.def_type(H_type, C_type, C_type, C_type, c0=0.0, c1=0.5, c2=0.0, c3=-0.25)


<DihedralType: HC-C3-C3-C3>

In [5]:
typifier = mp.typifier.Typifier(ff)
typifier.typify_bonds(ethane)

list(ethane.bonds)
ethane.bonds[0]["$type"]["r0"]

1.09

In [6]:
# create ethane mesh
offset = 3.5
structs = []
for i in range(2):
    for j in range(2):
        for k in range(2):
            eth = ethane.copy()
            eth.translate((i*offset, j*offset, k*offset))
            structs.append(eth)

frame = mp.Frame.from_structs(structs)
# mp.io.write_pdb("ethanes.pdb", frame)

In [7]:
frame["bonds"]["type"]

array([1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1,
       1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1,
       1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2])

In [8]:
class Ethane(mp.Monomer):
    def __init__(self, **props):
        super().__init__(**props)
        self["name"] = "ethane"

        # 构造两个 CH3
        ch3_1 = mp.builder.CH3()
        ch3_1.translate(-ch3_1["atoms"][0].xyz)  # 将碳原子移至原点

        ch3_2 = mp.builder.CH3()
        ch3_2.rotate(180, axis=(0, 1, 0))  # 反向
        ch3_2.translate((1.54, 0, 0))     # 典型 C-C 距离 = 0.154 A

        self.add_struct(ch3_1)
        self.add_struct(ch3_2)

        c1 = ch3_1["atoms"][0]
        c2 = ch3_2["atoms"][0]
        self.def_bond(c1, c2)

ethane = Ethane(name="ethane", net_charge=0)
ambertypifier = mp.typifier.AmberToolsTypifier("gaff", "bcc")
ambertypifier.typify(ethane, workdir="./amber")

Usage: antechamber -i     input file name
                   -fi    input file format
                   -o     output file name
                   -fo    output file format
                   -c     charge method
                   -cf    charge file name
                   -nc    net molecular charge (int)
                   -a     additional file name
                   -fa    additional file format
                   -ao    additional file operation
                          crd   : only read in coordinate
                          crg   : only read in charge
                          radius: only read in radius
                          name  : only read in atom name
                          type  : only read in atom type
                          bond  : only read in bond type 
                   -m     multiplicity (2S+1), default is 1
                   -rn    residue name, overrides input file, default is MOL
                   -rf    residue toplogy file name in prep input f

<Monomer: 8 atoms>

In [9]:
%load_ext autoreload
%autoreload 2

import molpy as mp

class Carbonyl(mp.Monomer):
    def __init__(self, **props):
        super().__init__(**props)
        self["name"] = "carbonyl"
        self.def_atom(name="C", type="c", element="C", xyz=(0, 0, 0))
        self.def_atom(name="O", type="o", element="O", xyz=(0, 0, 1.2))
        self.def_bond(0, 1)

class Methylene(mp.Monomer):
    def __init__(self, **props):
        super().__init__(**props)
        self["name"] = "methylene"
        self.def_atom(name="C", type="c", element="C", xyz=(0, 0, 0))
        self.def_atom(name="H1", type="h", element="H", xyz=(0, 0, 1.2))
        self.def_atom(name="H2", type="h", element="H", xyz=(0, 0, -1.2))
        self.def_bond(0, 1)
        self.def_bond(0, 2)

class Methyl(mp.Monomer):
    def __init__(self, **props):
        super().__init__(**props)
        self["name"] = "methyl"
        self.def_atom(name="C", type="c", element="C", xyz=(0, 0, 0))
        self.def_atom(name="H1", type="h", element="H", xyz=(0.000, 0.000, 1.090))
        self.def_atom(name="H2", type="h", element="H", xyz=(1.026, 0.000, -0.363))
        self.def_atom(name="H3", type="h", element="H", xyz=(-0.513, 0.889, -0.363))
        self.def_bond(0, 1)
        self.def_bond(0, 2)
        self.def_bond(0, 3)
        

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
head = Methyl()
body = Methylene()
tail = Carbonyl()
head.def_link_site(
    this=head["atoms"][0],
    that=body["atoms"][0],
    
)

amberreactor = mp.reactor.AmberToolsReactor()
amberreactor.react(head, workdir="./amber")

AttributeError: module 'molpy' has no attribute 'reactor'

In [ ]:
import random
n_monomer = 10
structs = []

head = prev = Methyl()
structs.append(head)

for i in range(1, n_monomer+1):

    # 随机选择一个单体
    if random.random() < 0.5:
        monomer = Carbonyl()
    else:
        monomer = Methylene()

    monomer.rotate(180*i, axis=(1, 0, 0)).translate([2.95*i, 0, 0])

    monomer.def_link_site(
        this=monomer["atoms"][0],
        that=prev["atoms"][0],
    )

    structs.append(monomer)

    prev = monomer


tail = Methyl()
tail.rotate(180, axis=(1, 0, 0)).translate([2.95*i, 0, 0])
tail.def_link_site(
    this=tail["atoms"][0],
    that=prev["atoms"][0],
)
structs.append(tail)

In [ ]:
peo_frame = mp.io.read_pdb("peo/PEO_initial.pdb")
peo_monomer = mp.Monomer.from_frame(peo_frame, "peo5")
ambertypifier = mp.typifier.AmberToolsTypifier("gaff", "bcc")
ambertypifier.typify(peo_monomer, workdir="./peo")

Info: acdoctor mode is on: check and diagnose problems in the input file.
Info: The atom type is set to gaff; the options available to the -at flag are
      gaff, gaff2, amber, bcc, abcg2, and sybyl.

-- Check Format for pdb File --
   Status: pass
Usage: antechamber -i     input file name
                   -fi    input file format
                   -o     output file name
                   -fo    output file format
                   -c     charge method
                   -cf    charge file name
                   -nc    net molecular charge (int)
                   -a     additional file name
                   -fa    additional file format
                   -ao    additional file operation
                          crd   : only read in coordinate
                          crg   : only read in charge
                          radius: only read in radius
                          name  : only read in atom name
                          type  : only read in atom type
            

<Monomer: 43 atoms>

In [ ]:
head = peo_monomer.clone(name="head")
body = peo_monomer.clone(name="body")
tail = peo_monomer.clone(name="tail")

head_id = 0
tail_id = 6
tail_omit = [2, 22, 23, 24]
head_omit = [7, 31, 32, 33]

head.def_link_site(
    this=head["atoms"][head_id],
    that=body["atoms"][tail_id],
    delete=[head["atoms"][i] for i in head_omit]
)
body.def_link_site(
    this=tail["atoms"][tail_id],
    that=body["atoms"][head_id],
    delete=[tail["atoms"][i] for i in tail_omit]
)
body.def_link_site(
    this=head["atoms"][head_id],
    that=body["atoms"][tail_id],
    delete=[head["atoms"][i] for i in head_omit]
)
tail.def_link_site(
    this=tail["atoms"][tail_id],
    that=body["atoms"][head_id],
    delete=[tail["atoms"][i] for i in tail_omit]
)

<Monomer: 43 atoms>

In [ ]:
amberreactor = mp.reacter.AmberToolsReactor()
amberreactor.react(head, workdir=Path("./peo"))
amberreactor.react(body, workdir=Path("./peo"))
amberreactor.react(tail, workdir=Path("./peo"))


 Cannot open the main chain file: peo/head/head.mc, exit
 Cannot open the main chain file: peo/tail/tail.mc, exit
 Cannot open the main chain file: peo/body/body.mc, exit

In [ ]:
peo = mp.Polymer([head, body, tail])
amberpolymerizer = mp.typifier.AmberToolsPolymerize()
amberpolymerizer.polymerize(peo, workdir=Path("./peo"))